## Examen Victor Marinas

## Partidos de Futbol:

## Inicializar spark context y cargar los datos desde el fichero.

In [1]:
from pyspark.sql import Row
from pyspark import SparkContext
import numpy as np
import pandas as pd
sc= SparkContext()



In [2]:
datos = sc.textFile("DataSetPartidos.txt")

In [3]:
# Observamos que los datos están separados por :: 

datos.take(5)

['1::1970-71::1::Athletic Club::Barcelona::1::1::12/09/1970::21942000.0',
 '2::1970-71::1::Las Palmas::Atletico de Madrid::1::1::12/09/1970::21942000.0',
 '3::1970-71::1::Real Madrid::Valencia::2::0::12/09/1970::21942000.0',
 '4::1970-71::1::Celta de Vigo::Sporting de Gijon::2::0::13/09/1970::22028400.0',
 '5::1970-71::1::Elche::Granada::1::1::13/09/1970::22028400.0']

In [12]:
#Realizamos el parseado eliminando la separacion por  ::

datos_split = datos.map(lambda x: x.split("::"))

datos_split.take(1)



[['1',
  '1970-71',
  '1',
  'Athletic Club',
  'Barcelona',
  '1',
  '1',
  '12/09/1970',
  '21942000.0']]

In [21]:
from  pyspark.sql import Row

#Importante aqui, cogemos la base de datos inicial, que no ha sido parseada, en este caso se llama datos

datos_futbol = datos.\
    map(lambda x: x.split("::")).\
    map(lambda x: Row( temporada = x[1],
                     jornada = x[2],
                     equipo_local = x[3],
                     equipo_visitante = x[4],
                     gol_local = int(x[5]),
                     gol_visit = int(x[6]) 
                     ))



# Importante en los numeros, poner int

In [23]:
datos_futbol.take(1)

[Row(equipo_local='Athletic Club', equipo_visitante='Barcelona', gol_local=1, gol_visit=1, jornada='1', temporada='1970-71')]

## Tercera pregunta: Calcular el número total de goles que ha marcado el Sporting de Gijón.

In [ ]:
# Habria que diferenciar los goles que han metido de Local y los de visitante:

# Creamos una tabla con los equipos local, y marcador local

#Filtramos por Sporting

In [32]:
sporting_local = datos_futbol.map(lambda x: (x[0],x[2])).filter(lambda x:'Sporting de Gijon' in x[0])

sporting_local.take(5)

[('Sporting de Gijon', 3),
 ('Sporting de Gijon', 0),
 ('Sporting de Gijon', 0),
 ('Sporting de Gijon', 3),
 ('Sporting de Gijon', 3)]

In [40]:
# Hacemos un sumatorio de los goles:


suma_local = sporting_local.reduceByKey(lambda x, y: x + y)

suma_local.take(5)




[('Sporting de Gijon', 1275)]

# Si nos preguntaran por la media por partido

In [50]:
# Primero seleccionamos la primera fila que contiene el Sporting como local.

# Hacemos un sumatorio de los partidos de local


count_partidos = sporting_local.map(lambda x: (x[0],1))

count_partidos_2 = count_partidos.reduceByKey(lambda x, y: x + y)

count_partidos_2.take(5)

[('Sporting de Gijon', 853)]

# Juntariamos los dos datasets con la funcion Join

In [52]:
join = suma_local.join(count_partidos_2)

join.take(5)

[('Sporting de Gijon', (1275, 853))]

In [55]:
media_local = join.map(lambda x: (x[0], round(x[1][0]/x[1][1],2)))

media_local.take(1)

[('Sporting de Gijon', 1.49)]


# 1.49 goles marca el Sporting de Gijón de Media como local.

In [ ]:
# Creamos una tabla con los equipos visitantes, y marcador visitante

In [39]:
sporting_visit = datos_futbol.map(lambda x: (x[1],x[3]))


sporting_visit.take(5)




[('Barcelona', 1),
 ('Atletico de Madrid', 1),
 ('Valencia', 0),
 ('Sporting de Gijon', 0),
 ('Granada', 1)]

In [42]:
#Filtramos por Sporting

#Ponemos 0, porque ahora hemos creado un nuevo dataframe 0 nombre equipo y 1 goles visitante

sporting_visit_2 = sporting_visit.filter(lambda x: "Sporting de Gijon" in x[0])


sporting_visit_2.take(5)


[('Sporting de Gijon', 0),
 ('Sporting de Gijon', 0),
 ('Sporting de Gijon', 1),
 ('Sporting de Gijon', 1),
 ('Sporting de Gijon', 1)]

In [43]:
goles_visit = sporting_visit_2.reduceByKey(lambda x, y: x + y)

goles_visit.take(5)

[('Sporting de Gijon', 767)]

**En total ha metido 1275 goles de local y 767 de visitante**

**En total 2042 goles**

## Cuarta pregunta: ¿ En que temporada se marcaron más goles?

In [ ]:
# Creamos una variable que se va a llamar goles_partido

In [56]:
datos_futbol.take(1)

[Row(equipo_local='Athletic Club', equipo_visitante='Barcelona', gol_local=1, gol_visit=1, jornada='1', temporada='1970-71')]

In [181]:
datos_futbol_2 = datos.\
    map(lambda x: x.split("::")).\
    map(lambda x: Row( temporada = str(x[1]),
                     jornada = x[2],
                     equipo_local = x[3],
                     equipo_visitante = x[4],
                     gol_local = int(x[5]),
                     gol_visit = int(x[6]),
                     goles_partido = int(x[5])+int(x[6]) 
                     ))
    
datos_futbol_2.take(1)    

[Row(equipo_local='Athletic Club', equipo_visitante='Barcelona', gol_local=1, gol_visit=1, goles_partido=2, jornada='1', temporada='1970-71')]

In [182]:
# Hemos seleccionado los goles de  local y visitante y las temporadas


goles_temporada = datos_futbol_2.map(lambda x: (x[6],x[5]))

goles_temporada.take(5)

[('1970-71', '1'),
 ('1970-71', '1'),
 ('1970-71', '1'),
 ('1970-71', '1'),
 ('1970-71', '1')]

In [189]:
def get_temporada(x):
    return x[5:7] + " Temporada"

In [190]:
get_temporada("1974-75")

'75 Temporada'

In [192]:
goles_temporada = datos_futbol_2.map(lambda x: (x[6],x[5]))

goles_temporada.take(5)

[('1970-71', '1'),
 ('1970-71', '1'),
 ('1970-71', '1'),
 ('1970-71', '1'),
 ('1970-71', '1')]

In [194]:
temporada_goleadora = goles_temporada.\
                    map(lambda x: (get_temporada(x[0]), x[1]))
    
    
temporada_goleadora.take(5)

[('71 Temporada', '1'),
 ('71 Temporada', '1'),
 ('71 Temporada', '1'),
 ('71 Temporada', '1'),
 ('71 Temporada', '1')]

In [196]:
temporada_goleadora_2 = temporada_goleadora.reduceByKey(lambda x, y: x + y)



    
temporada_goleadora_2.take(5)

[('92 Temporada',
  '1111111111222222222233333333334444444444555555555566666666667777777777888888888899999999991010101010101010101011111111111111111111121212121212121212121313131313131313131314141414141414141414151515151515151515151616161616161616161617171717171717171717181818181818181818181919191919191919191920202020202020202020212121212121212121212222222222222222222223232323232323232323242424242424242424242525252525252525252526262626262626262626272727272727272727272828282828282828282829292929292929292929303030303030303030303131313131313131313132323232323232323232333333333333333333333434343434343434343435353535353535353535363636363636363636363737373737373737373738383838383838383838111111111122222222223333333333444444444455555555556666666666777777777788888888889999999999101010101010101010101111111111111111111112121212121212121212131313131313131313131414141414141414141415151515151515151515161616161616161616161717171717171717171718181818181818181818191919191919191919192020202020202020202

In [ ]:
datos_decadas_ordenados = datos_lista_temporada.\
    map(lambda x: (get_decade(x[0]),x[1])).\
    reduceByKey(lambda x,y : x+y).\
    sortBy(lambda x: -x[1])
    
decadas_goleadoras = datos_decadas_ordenados.\
    take(3)

In [191]:
goles_temporada_2 = goles_temporada.reduceByKey(lambda x, y : x + y )

goles_temporada_2.take(5)

[('1974-75',
  '111111111222222222333333333444444444555555555666666666777777777888888888999999999101010101010101010111111111111111111121212121212121212131313131313131313141414141414141414151515151515151515161616161616161616171717171717171717181818181818181818191919191919191919202020202020202020212121212121212121222222222222222222232323232323232323242424242424242424252525252525252525262626262626262626272727272727272727282828282828282828292929292929292929303030303030303030313131313131313131323232323232323232333333333333333333343434343434343434111111111122222222223333333333444444444455555555556666666666777777777788888888889999999999101010101010101010101111111111111111111112121212121212121212131313131313131313131414141414141414141415151515151515151515161616161616161616161717171717171717171718181818181818181818191919191919191919192020202020202020202021212121212121212121222222222222222222222323232323232323232324242424242424242424252525252525252525252626262626262626262627272727272727272727282

In [83]:

# Arriba Preguntar

In [ ]:
equipo_local = local.reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] > 1400)


equipo_local.take(20)

## Quinta pregunta: ¿Cúal es el equipo que tiene el record de más goles como local? ¿ y cómo visitante?

In [84]:
# Elegimos los equipos locales y goles local


local = datos_futbol.map(lambda x: (x[0], x[2]))

local.take(5)

[('Athletic Club', 1),
 ('Las Palmas', 1),
 ('Real Madrid', 2),
 ('Celta de Vigo', 2),
 ('Elche', 1)]

In [107]:
# Sumamos con reduceByKey los goles por equipo


# Y filtramos por goles mayores de 1400, para ver los más goleadores



equipo_local = local.reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] > 1400)


equipo_local.take(20)

[('Athletic Club', 1424),
 ('Barcelona', 2043),
 ('Real Madrid', 2054),
 ('Valencia', 1522),
 ('Atletico de Madrid', 1566)]

# Equipo más goleador como local es el Real Madrid con 2054 goles

In [ ]:
# Elegimos los equipos visitantes y goles visitantes

In [108]:

visitante = datos_futbol.map(lambda x: (x[1], x[3]))

visitante.take(5)

[('Barcelona', 1),
 ('Atletico de Madrid', 1),
 ('Valencia', 0),
 ('Sporting de Gijon', 0),
 ('Granada', 1)]

In [110]:
equipo_visitante = visitante.reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] > 1000)


equipo_visitante.take(20)

[('Barcelona', 1296), ('Real Madrid', 1296)]

# Como Visitante el Madrid y el Barça empatan a 1296 goles

## Sexta pregunta: ¿Cúales son las 3 décadas en las que más goles se metieron?

##  Séptima pregunta: ¿Cúal es la media de victorias de los equipos que han estado menos de 10 temporadas en primera división?

##  Octava pregunta: ¿Cúal es la media de goles como visitante por partido?

In [111]:
datos_futbol.take(1)

[Row(equipo_local='Athletic Club', equipo_visitante='Barcelona', gol_local=1, gol_visit=1, jornada='1', temporada='1970-71')]

In [165]:
# Revisar


goles_partido_visitante = datos_futbol.map(lambda x: (x[0],x[3]))


goles_partido_visitante.take(5)

[('Athletic Club', 1),
 ('Las Palmas', 1),
 ('Real Madrid', 0),
 ('Celta de Vigo', 0),
 ('Elche', 1)]

In [160]:
total_goles_visitante = goles_partido_visitante.reduceByKey(lambda x, y: x + y)

In [161]:
# Obtenemos los goles por temporada, siendo visitante

total_goles_visitante.take(20)

[('Athletic Club', 794),
 ('Almeria', 335),
 ('Orihuela', 15),
 ('Moscardo', 12),
 ('Cartagena', 99),
 ('Tenerife', 674),
 ('Palamos', 101),
 ('Linares CF', 78),
 ('Sevilla', 734),
 ('Leganes', 236),
 ('Hercules', 574),
 ('Sevilla Atletico', 60),
 ('Figueres', 118),
 ('Sporting de Gijon', 816),
 ('Girona', 138),
 ('Lorca Deportiva', 45),
 ('CD Malaga', 326),
 ('Mallorca B', 27),
 ('Extremadura', 139),
 ('Real Union', 24)]

In [163]:
# Necesitamos saber el numero de partidos  que se jugaron en total:


# Vemos el numero de partidos que se jugaron por temporada

numero_partidos = datos_futbol.map(lambda x: (x[5],1))

numero_partidos_2 = numero_partidos.reduceByKey(lambda x, y: x + y)


numero_partidos_2.take(80)




[('1974-75', 686),
 ('1979-80', 686),
 ('1996-97', 842),
 ('2000-01', 842),
 ('2004-05', 842),
 ('1972-73', 686),
 ('1991-92', 760),
 ('1994-95', 760),
 ('2003-04', 842),
 ('1984-85', 686),
 ('1986-87', 795),
 ('1970-71', 620),
 ('1971-72', 686),
 ('1997-98', 842),
 ('2009-10', 842),
 ('2014-2015', 735),
 ('1989-90', 760),
 ('1987-88', 760),
 ('1985-86', 686),
 ('1982-83', 686),
 ('1992-93', 760),
 ('1981-82', 686),
 ('1998-99', 842),
 ('2011-12', 842),
 ('2007-08', 842),
 ('1973-74', 686),
 ('2001-02', 842),
 ('1977-78', 686),
 ('2010-11', 842),
 ('1975-76', 686),
 ('2006-07', 842),
 ('1980-81', 686),
 ('1995-96', 842),
 ('2013-14', 842),
 ('1999-00', 842),
 ('2008-09', 842),
 ('1993-94', 760),
 ('1978-79', 686),
 ('2012-13', 842),
 ('2005-06', 842),
 ('2002-03', 842),
 ('1983-84', 686),
 ('1988-89', 760),
 ('1990-91', 760),
 ('1976-77', 686)]

In [166]:


datos_futbol.count()

34518